In [ ]:
## author: Andrew (Jake) Shullaw 8/13/2021

## Validation(ish) Set
-   In this notebook, we will somewhat validate our models against a validation set.  It is comprised of both real and fake data.  The problem is that we do not have a confirmation on which files are real and fake or if the real ones would be approved.  *C'est la vie*.
-   Either way, the purpose of this file is to apply the same data transformations to the validation set as we did the train and test sets.
- To save some time on this 60,000 report data set, I've saved the file once I parsed it so that you do not have to :D

In [2]:
# if you have the validation csv, do not run this block of code

# import pathlib
# import json
# import pandas
# import numpy as np

# path = pathlib.Path.home()
# path = path.joinpath('Documents','CGI','CAER','jupyter','jsons_with_comments')
# data = path.glob('*')
# data_list = [x for x in data if x.suffix == '.json']

In [4]:
# if you have the validation csv, do not run this block of code

# jsons = path.glob('*.json')
# jsons_list = [str(x) for x in jsons if x.is_file()]
# e_list = []
# for file in jsons_list:
#     with open(file, 'r', encoding='utf-8') as json_file:
#         i = json.load(json_file)['id']
#     with open(file, 'r', encoding='utf-8') as json_file:
#         e_table = json.load(json_file)['emissions']
#         for e in e_table:         
#             e_list.append(e)
#             e_list.append(i)

In [5]:
# if you have the validation csv, do not run this block of code

# df = pandas.DataFrame(data=e_list[::2])
# df.rename(columns={'id':'eid'}, inplace=True)
# df.drop('_row',axis=1,inplace=True)
# df.drop('sheetName',axis=1,inplace=True)
# df.insert(0,'id',e_list[1::2])  

In [208]:
# if you have the validation csv, do not run this block of code


# df.to_csv('jsonswithcomments_EMISSIONS_DATAFRAME.csv')

In [2]:
import pandas

# run if you have validation csv

df = pandas.read_csv('jsonswithcomments_EMISSIONS_DATAFRAME.csv')  # validation csv
df.drop('Unnamed: 0', axis=1, inplace=True)

Clean validation

In [3]:
import numpy as np

description = df.describe(include='all')           #
nonzero_cols = [d for d in description if description[d]['count'] > 0]
df = df[nonzero_cols]
df
def type_convert(df):
    df['id'] = df['id'].astype('string')
    df['eid'] = df['eid'].astype('string')
    df['displayName'] = df['displayName'].astype('category')
    df['pollutantCode'] = df['pollutantCode'].astype('category')
    df['overallControlPercent'] = df['overallControlPercent'].astype('float')
    df['totalEmissions'] = df['totalEmissions'].astype('float')
    df['emissionsUomCode'] = df['emissionsUomCode'].astype('category')
    df['emissionsFactor'] = df['emissionsFactor'].astype('float')
    df['emissionsFactorFormula'] = df['emissionsFactorFormula'].astype('string')
    df['emissionsFactorText'] = df['emissionsFactorText'].astype('string')
    df['emissionsCalcMethodCode'] = df['emissionsCalcMethodCode'].astype('category')
    df['comments'] = df['comments'].astype('string')
    df['calculationComment'] = df['calculationComment'].astype('string')
    df['emissionsNumeratorUom'] = df['emissionsNumeratorUom'].astype('category')
    df['emissionsDenominatorUom'] = df['emissionsDenominatorUom'].astype('category')
    df['calculatedEmissionsTons'] = df['calculatedEmissionsTons'].astype('float')
    return df
df = type_convert(df)
df.dtypes
df.fillna(value=np.nan, inplace=True)  # replace all NoneType with Nan
df.dropna(axis=0, how='all', inplace=True) # drop empty/nan columns/rows
df.dropna(axis=1, how='all', inplace=True) # drop empty/nan columns/rows
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)   # replace all empty cells with NaN value
df
def convert_compounds(df):
    names = ['Nitrogen', 'Carbon', 'Volatile Organic Compounds', 'Sulfur', 'Filt + Cond', 'Filterable',
            'Ammonia', 'Condensible', 'Lead', 'Styrene', 'Ether', 'Mercury', 'Xylenes', 'Cumene', 'Tetrachloroethylene',
            'Formaldehyde', 'Methanol', 'Acetal']
    for n in names:
        df.displayName = df.displayName.str.replace(r'.+'+n+'.+', n)
        if n=='Filt + Cond':
            df.displayName = df.displayName.str.replace(r'.+' + "Filt \+ Cond" + '.+', n)
        if n=='Volatile Organic Compounds':
            df.displayName = df.displayName.str.replace(r'.+'+n+'.+', 'VOC')
    names[2] = 'VOC'
    return df
df = convert_compounds(df)
df.displayName.value_counts()
total = len(df)
nan_columns = [d for d in df if df[d].isna().sum()/total > .9]
# new_df.drop(nan_columns, axis=1, inplace=True)
print(nan_columns)
# new_df.describe(include='all')
nan_ratios = {df[d].isna().sum()/total:d for d in df}
nan_ratios
df.drop(['reportingPeriodId','overallControlPercent','emissionsFactorFormula'], axis=1, inplace=True)
# rewriting function (removing dropped columns)
def convert(edf : pandas.DataFrame): 
    edf['id'] = edf['id'].astype('string')
    edf['eid'] = edf['eid'].astype('string')
    edf['displayName'] = edf['displayName'].astype('category')
    edf['emissionsFactor'] = edf['emissionsFactor'].astype('float')
    edf['emissionsFactorText'] = edf['emissionsFactorText'].astype('string')
    edf['emissionsCalcMethodCode'] = edf['emissionsCalcMethodCode'].astype('category')
    edf['comments'] = edf['comments'].astype('string')
    edf['calculationComment'] = edf['calculationComment'].astype('string')
    edf['calculatedEmissionsTons'] = edf['calculatedEmissionsTons'].astype('float')
    return edf
edf = convert(df)
edf.drop('pollutantCode', axis=1, inplace=True)
edf.comments = edf.comments.str.replace(r'Comment.+', 'None')  #replacing all " Comment from xxxx: "
description = edf.describe(include='all')
description.loc['type'] = edf.dtypes
edf.comments.fillna(value="None", inplace=True)
edf = convert(edf)
description
edf.drop(['emissionsUomCode','emissionsNumeratorUom', 'emissionsDenominatorUom', 'totalEmissions'], axis=1, inplace=True)

['overallControlPercent', 'emissionsFactorFormula', 'calculationComment']


In [4]:
edf.dtypes

id                           string
eid                          string
displayName                category
totalManualEntry               bool
emissionsFactor             float64
emissionsFactorText          string
emissionsCalcMethodCode    category
comments                     string
calculationComment           string
calculatedEmissionsTons     float64
dtype: object

Encode validation

In [6]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
X = edf
X_cats = X.select_dtypes('category')
encoder.fit(X_cats)
codes = encoder.transform(X_cats).toarray()
feature_names = encoder.get_feature_names(['displayName', 'emissionsCalcMethodCode'])

X = pandas.concat([edf.select_dtypes(exclude='category'),
                   pandas.DataFrame(codes, columns=feature_names).astype(int)], axis=1)
X

,id,eid,totalManualEntry,emissionsFactor,emissionsFactorText,comments,calculationComment,calculatedEmissionsTons,"displayName_001-001-Annual(1,1,2,2-Tetrachloroethane)","displayName_001-001-Annual(2,2,4-Trimethylpentane)",...,emissionsCalcMethodCode_24,emissionsCalcMethodCode_28,emissionsCalcMethodCode_29,emissionsCalcMethodCode_30,emissionsCalcMethodCode_31,emissionsCalcMethodCode_32,emissionsCalcMethodCode_33,emissionsCalcMethodCode_40,emissionsCalcMethodCode_41,emissionsCalcMethodCode_42
0,1001,67351,False,0.0001,"Emission Factor for PM Filt 4.63E-05, PM10 Fil...","Emission factor per AP-42 Section 13.2.4, Aggr...",<NA>,0.0053,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,67352,False,0.0010,(usage * factor/2000)*(1-control efficiency/100),None,<NA>,8.7100,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1001,67354,False,NaN,<NA>,None,<NA>,328.5000,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1001,67353,False,NaN,<NA>,None,<NA>,2050.3000,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1001,67357,False,NaN,<NA>,None,<NA>,2.4500,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59155,999,67327,False,5.7000,<NA>,None,<NA>,0.0500,0,0,...,0,0,0,0,0,0,0,0,0,0
59156,999,67331,False,1.9000,<NA>,None,<NA>,0.0200,0,0,...,0,0,0,0,0,0,0,0,0,0
59157,999,67325,False,7.6000,<NA>,None,<NA>,0.0700,0,0,...,0,0,0,0,0,0,0,0,0,0
59158,999,67326,False,0.6000,<NA>,None,<NA>,0.0100,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
cols = ['id', 'eid', 'totalManualEntry', 'emissionsFactor',
       'calculatedEmissionsTons', 'displayName_Acetal', 'displayName_Ammonia',
       'displayName_Carbon', 'displayName_Condensible', 'displayName_Cumene',
       'displayName_Ether', 'displayName_Filt + Cond',
       'displayName_Filterable', 'displayName_Formaldehyde',
       'displayName_Lead', 'displayName_Mercury', 'displayName_Methanol',
       'displayName_Nitrogen', 'displayName_Styrene', 'displayName_Sulfur',
       'displayName_Tetrachloroethylene',
       'displayName_Volatile Organic Compounds', 'displayName_Xylenes',
       'emissionsCalcMethodCode_1', 'emissionsCalcMethodCode_10',
       'emissionsCalcMethodCode_11', 'emissionsCalcMethodCode_12',
       'emissionsCalcMethodCode_13', 'emissionsCalcMethodCode_2',
       'emissionsCalcMethodCode_24', 'emissionsCalcMethodCode_28',
       'emissionsCalcMethodCode_29', 'emissionsCalcMethodCode_3',
       'emissionsCalcMethodCode_30', 'emissionsCalcMethodCode_31',
       'emissionsCalcMethodCode_32', 'emissionsCalcMethodCode_33',
       'emissionsCalcMethodCode_4', 'emissionsCalcMethodCode_41',
       'emissionsCalcMethodCode_42', 'emissionsCalcMethodCode_5',
       'emissionsCalcMethodCode_7', 'emissionsCalcMethodCode_8',
       'emissionsCalcMethodCode_9']
X = X[cols]
X

,id,eid,totalManualEntry,emissionsFactor,calculatedEmissionsTons,displayName_Acetal,displayName_Ammonia,displayName_Carbon,displayName_Condensible,displayName_Cumene,...,emissionsCalcMethodCode_31,emissionsCalcMethodCode_32,emissionsCalcMethodCode_33,emissionsCalcMethodCode_4,emissionsCalcMethodCode_41,emissionsCalcMethodCode_42,emissionsCalcMethodCode_5,emissionsCalcMethodCode_7,emissionsCalcMethodCode_8,emissionsCalcMethodCode_9
0,1001,67351,False,0.0001,0.0053,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1001,67352,False,0.0010,8.7100,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1001,67354,False,NaN,328.5000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1001,67353,False,NaN,2050.3000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1001,67357,False,NaN,2.4500,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59155,999,67327,False,5.7000,0.0500,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
59156,999,67331,False,1.9000,0.0200,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
59157,999,67325,False,7.6000,0.0700,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
59158,999,67326,False,0.6000,0.0100,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
X.isna().sum()

id                                            0
eid                                           0
totalManualEntry                              0
emissionsFactor                           19219
calculatedEmissionsTons                       2
displayName_Acetal                            0
displayName_Ammonia                           0
displayName_Carbon                            0
displayName_Condensible                       0
displayName_Cumene                            0
displayName_Ether                             0
displayName_Filt + Cond                       0
displayName_Filterable                        0
displayName_Formaldehyde                      0
displayName_Lead                              0
displayName_Mercury                           0
displayName_Methanol                          0
displayName_Nitrogen                          0
displayName_Styrene                           0
displayName_Sulfur                            0
displayName_Tetrachloroethylene         

In [9]:
X.fillna(value=0, inplace=True)  # replace all NoneType with Nan
X.isna().sum()

C:\Users\ASHULLAW\AppData\Local\Programs\PythonCodingPack\lib\site-packages\pandas\core\frame.py:4311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


id                                        0
eid                                       0
totalManualEntry                          0
emissionsFactor                           0
calculatedEmissionsTons                   0
displayName_Acetal                        0
displayName_Ammonia                       0
displayName_Carbon                        0
displayName_Condensible                   0
displayName_Cumene                        0
displayName_Ether                         0
displayName_Filt + Cond                   0
displayName_Filterable                    0
displayName_Formaldehyde                  0
displayName_Lead                          0
displayName_Mercury                       0
displayName_Methanol                      0
displayName_Nitrogen                      0
displayName_Styrene                       0
displayName_Sulfur                        0
displayName_Tetrachloroethylene           0
displayName_Volatile Organic Compounds    0
displayName_Xylenes             

In [10]:
# X.drop('comments', axis=1, inplace=True)
# X['calculationComment'].fillna(value='0', inplace=True)
X.describe(include='all')

,id,eid,totalManualEntry,emissionsFactor,calculatedEmissionsTons,displayName_Acetal,displayName_Ammonia,displayName_Carbon,displayName_Condensible,displayName_Cumene,...,emissionsCalcMethodCode_31,emissionsCalcMethodCode_32,emissionsCalcMethodCode_33,emissionsCalcMethodCode_4,emissionsCalcMethodCode_41,emissionsCalcMethodCode_42,emissionsCalcMethodCode_5,emissionsCalcMethodCode_7,emissionsCalcMethodCode_8,emissionsCalcMethodCode_9
count,59160,59160,59160,59160.000000,5.916000e+04,59160.000000,59160.000000,59160.000000,59160.000000,59160.000000,...,59160.000000,59160.000000,59160.000000,59160.000000,59160.000000,59160.000000,59160.000000,59160.000000,59160.000000,59160.000000
unique,820,59160,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,3269,12547,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1605,1,58850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,338.420322,4.017511e+02,0.000794,0.015331,0.103854,0.037204,0.000270,...,0.000913,0.001572,0.001285,0.083249,0.004564,0.000051,0.008925,0.024003,0.467444,0.010176
std,NaN,NaN,NaN,6197.766825,3.023213e+04,0.028175,0.122868,0.305074,0.189263,0.016443,...,0.030199,0.039618,0.035819,0.276260,0.067403,0.007121,0.094050,0.153059,0.498943,0.100361
min,NaN,NaN,NaN,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,0.000000,9.000000e-03,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,NaN,NaN,NaN,0.050000,1.375980e-01,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,NaN,NaN,NaN,1.900000,1.470000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000


In [11]:
X['id'].isna().values.any()

False

In [12]:
X['totalManualEntry'] = X['totalManualEntry'].astype(int)
# X.drop(['eid'], axis=1, inplace=True)
# X.fillna(value=0, inplace=True)
X.isna().values.any()  # check entire DF for nan values

<ipython-input-12-d86f2473bd68>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['totalManualEntry'] = X['totalManualEntry'].astype(int)


False

In [13]:
X

,id,eid,totalManualEntry,emissionsFactor,calculatedEmissionsTons,displayName_Acetal,displayName_Ammonia,displayName_Carbon,displayName_Condensible,displayName_Cumene,...,emissionsCalcMethodCode_31,emissionsCalcMethodCode_32,emissionsCalcMethodCode_33,emissionsCalcMethodCode_4,emissionsCalcMethodCode_41,emissionsCalcMethodCode_42,emissionsCalcMethodCode_5,emissionsCalcMethodCode_7,emissionsCalcMethodCode_8,emissionsCalcMethodCode_9
0,1001,67351,0,0.0001,0.0053,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1001,67352,0,0.0010,8.7100,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1001,67354,0,0.0000,328.5000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1001,67353,0,0.0000,2050.3000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1001,67357,0,0.0000,2.4500,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59155,999,67327,0,5.7000,0.0500,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
59156,999,67331,0,1.9000,0.0200,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
59157,999,67325,0,7.6000,0.0700,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
59158,999,67326,0,0.6000,0.0100,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## Abandoned Cleaning of Compound Names
(collapse everything under this section (click to chevron))

In [219]:
annual = X.filter(like='displayName').filter(like='Annual').columns.to_list()
new_annual = pandas.Series(annual).str.replace(r'(displayName).+(Annual)','')
new_annual = new_annual.to_list()
dictionary = dict(zip(annual, new_annual))
X.rename(columns=dictionary, inplace=True)

In [220]:
import re
t = X.filter(like='Tetrachloroethane').columns.to_list()
new_t = pandas.Series(t).str.replace(r'.+(Tetrachloroethane)\)',r'\1', regex=True)
new_t = new_t.to_list()
dictionary = dict(zip(t, new_t))
X.rename(columns=dictionary, inplace=True)

In [221]:
X.columns.value_counts()[X.columns.value_counts() < 4]

(1,4-Dichlorobenzene)                                3
(Hydroquinone)                                       2
(Radionuclides (Radioactivity - Including Radon))    2
(Dibenzo[a,h]Anthracene)                             2
(Benzo[a]Pyrene)                                     2
                                                    ..
displayName_Methanol                                 1
emissionsCalcMethodCode_30                           1
displayName_Xylenes                                  1
eid                                                  1
emissionsCalcMethodCode_10                           1
Length: 67, dtype: int64

In [222]:
X2 = X.groupby(X.columns, axis=1).sum()

In [223]:
X2.columns

Index(['(1,1,2-Trichloroethane)', '(1,2,4-Trichlorobenzene)',
       '(1,2-Dibromo-3-Chloropropane)', '(1,3-Butadiene)',
       '(1,3-Dichloropropene)', '(1,4-Dichlorobenzene)',
       '(1-Methylnaphthalene)', '(2,2,4-Trimethylpentane)',
       '(2,4-Dinitrotoluene)', '(2-Chloroacetophenone)',
       ...
       'emissionsCalcMethodCode_41', 'emissionsCalcMethodCode_42',
       'emissionsCalcMethodCode_5', 'emissionsCalcMethodCode_6',
       'emissionsCalcMethodCode_7', 'emissionsCalcMethodCode_8',
       'emissionsCalcMethodCode_9', 'emissionsFactor', 'id',
       'totalManualEntry'],
      dtype='object', length=123)

In [224]:
df.columns

Index(['id', 'eid', 'displayName', 'totalManualEntry', 'emissionsFactor',
       'emissionsFactorText', 'emissionsCalcMethodCode', 'comments',
       'calculationComment', 'calculatedEmissionsTons'],
      dtype='object')

## Continue On....
## Scale the Novelty Data

In [14]:
scalers = X[['calculatedEmissionsTons','emissionsFactor']]

In [15]:
from sklearn.preprocessing import MinMaxScaler
X_scaled = X.copy()
scalers = MinMaxScaler().fit_transform(scalers.values)
X_scaled[['calculatedEmissionsTons','emissionsFactor']] = scalers
X_scaled.fillna(value=0, inplace=True)
X_scaled

,id,eid,totalManualEntry,emissionsFactor,calculatedEmissionsTons,displayName_Acetal,displayName_Ammonia,displayName_Carbon,displayName_Condensible,displayName_Cumene,...,emissionsCalcMethodCode_31,emissionsCalcMethodCode_32,emissionsCalcMethodCode_33,emissionsCalcMethodCode_4,emissionsCalcMethodCode_41,emissionsCalcMethodCode_42,emissionsCalcMethodCode_5,emissionsCalcMethodCode_7,emissionsCalcMethodCode_8,emissionsCalcMethodCode_9
0,1001,67351,0,8.183306e-10,1.053678e-09,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1001,67352,0,8.183306e-09,1.731610e-06,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1001,67354,0,0.000000e+00,6.530815e-05,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1001,67353,0,0.000000e+00,4.076143e-04,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1001,67357,0,0.000000e+00,4.870775e-07,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59155,999,67327,0,4.664484e-05,9.940358e-09,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
59156,999,67331,0,1.554828e-05,3.976143e-09,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
59157,999,67325,0,6.219313e-05,1.391650e-08,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
59158,999,67326,0,4.909984e-06,1.988072e-09,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [39]:
X[X.calculatedEmissionsTons>5000000]

,id,eid,totalManualEntry,emissionsFactor,calculatedEmissionsTons,displayName_Acetal,displayName_Ammonia,displayName_Carbon,displayName_Condensible,displayName_Cumene,...,emissionsCalcMethodCode_31,emissionsCalcMethodCode_32,emissionsCalcMethodCode_33,emissionsCalcMethodCode_4,emissionsCalcMethodCode_41,emissionsCalcMethodCode_42,emissionsCalcMethodCode_5,emissionsCalcMethodCode_7,emissionsCalcMethodCode_8,emissionsCalcMethodCode_9
37768,3252,162009,0,0.0,5030000.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
pandas.DataFrame([X_scaled.calculatedEmissionsTons,X.calculatedEmissionsTons]).T.describe()

,calculatedEmissionsTons,calculatedEmissionsTons
count,5.916000e+04,5.916000e+04
mean,7.987100e-05,4.017511e+02
std,6.010364e-03,3.023213e+04
min,0.000000e+00,0.000000e+00
25%,1.789264e-09,9.000000e-03
50%,2.735547e-08,1.375980e-01
75%,2.922465e-07,1.470000e+00
max,1.000000e+00,5.030000e+06


In [17]:
pandas.DataFrame([X_scaled.emissionsFactor,X.emissionsFactor]).T.describe()

,emissionsFactor,emissionsFactor
count,5.916000e+04,59160.000000
mean,2.769397e-03,338.420322
std,5.071822e-02,6197.766825
min,0.000000e+00,0.000000
25%,0.000000e+00,0.000000
50%,4.091653e-07,0.050000
75%,1.554828e-05,1.900000
max,1.000000e+00,122200.000000


In [21]:
from sklearn.svm import OneClassSVM
import pickle
# load
with open('OCSVM.pkl', 'rb') as f:  # open OCSVM model
    svm = pickle.load(f)
feats = ['emissionsCalcMethodCode_8',
 'emissionsCalcMethodCode_8',
 'displayName_Filt + Cond',
 'displayName_Volatile Organic Compounds',
 'displayName_Nitrogen',
 'emissionsCalcMethodCode_13',
 'displayName_Carbon',
 'displayName_Sulfur',
 'emissionsCalcMethodCode_2',
 'emissionsCalcMethodCode_10',
 'emissionsCalcMethodCode_3',
 'emissionsCalcMethodCode_4',
 'emissionsCalcMethodCode_1',
 'emissionsCalcMethodCode_7',
 'displayName_Lead',
 'emissionsCalcMethodCode_11',
 'displayName_Condensible',
 'displayName_Ammonia',
 'emissionsCalcMethodCode_30',
 'emissionsCalcMethodCode_28',
 'emissionsCalcMethodCode_41',
 'emissionsCalcMethodCode_5',
 'emissionsCalcMethodCode_9',
 'emissionsCalcMethodCode_29',
 'emissionsCalcMethodCode_33',
 'emissionsCalcMethodCode_24',
 'emissionsCalcMethodCode_31',
 'emissionsCalcMethodCode_32',
 'emissionsCalcMethodCode_42',
 'calculatedEmissionsTons',
 'displayName_Methanol',
 'emissionsFactor']
x = X_scaled[feats]
fake_predict = svm.predict(x)
n_error = fake_predict[fake_predict == -1].size
# # filter outlier index
error_idx = np.where(fake_predict == -1)
# # filter outlier values
error_values = X[feats].iloc[error_idx]

"Errors" or reports that were identified outside of the decision boundary

In [22]:
n_error

22333

This was the outlandish emissions file with 5 million tons of calculated emissions

In [20]:
error_values.loc[37768]

emissionsCalcMethodCode_8                       0.0
emissionsCalcMethodCode_8                       0.0
displayName_Filt + Cond                         0.0
displayName_Volatile Organic Compounds          0.0
displayName_Nitrogen                            0.0
emissionsCalcMethodCode_13                      0.0
displayName_Carbon                              1.0
displayName_Sulfur                              0.0
emissionsCalcMethodCode_2                       0.0
emissionsCalcMethodCode_10                      0.0
emissionsCalcMethodCode_3                       0.0
emissionsCalcMethodCode_4                       0.0
emissionsCalcMethodCode_1                       1.0
emissionsCalcMethodCode_7                       0.0
displayName_Lead                                0.0
emissionsCalcMethodCode_11                      0.0
displayName_Condensible                         0.0
displayName_Ammonia                             0.0
emissionsCalcMethodCode_30                      0.0
emissionsCal

In [23]:
# load
with open('IF.pkl', 'rb') as f:  # open Isolation Forest model
    if_clf = pickle.load(f)

In [24]:
from sklearn.ensemble import IsolationForest


y_predicted_test = if_clf.predict(x)

n_error_test = y_predicted_test[y_predicted_test == -1].size
# # filter outlier index
error_test_idx = np.where(y_predicted_test == -1)
# # filter outlier values
error_test_values = X.iloc[error_test_idx]

test_rate = round(n_error_test/len(x),2) * 100
print("errors: " +str(n_error_test) +  "/" + str(len(x)) + "("+str(test_rate)+"%)")    

errors: 4005/59160(7.000000000000001%)


One of these models is performing better.  We just cannot say which one because we do not know enough about the validation set.  Testing these models on reports that have been confirmed as denied because of their emissions table will give a more precise validation of the models.